## Roll number: 


### Instructions
 * Fill in the roll-number in the cell above.
 * Code must be submitted in Python in jupyter notebooks. We highly recommend using anaconda/miniconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 * For this assignment, you will be using Open3D  extensively. Refer to [Open3D](http://www.open3d.org/docs/release/) documentation.
 *  Most of the questions require you to **code your own functions** unless there is a need to call in the abilities of the mentioned libraries, such as Visualisation from Open3D. Make sure your code is modular since you will be reusing them for future assignments. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 *  All the representations are expected to be in a right-hand coordinate system.
<!--  * Answer to the descriptive questions should be answered in your own words. Copy-paste answers will lead to penalty. -->
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.
 * Commit the notebooks in the repo and any other results files under the result folder in the GitHub Classroom repo. 
 * Commits past the deadline will not be considered.
 * This is a group assignment. Discussions are encouraged but any sharing of code among different teams will be penalized. 

### Instructions for group formation
 * We have circulated google sheet in moodle to fill in team members. Please finalize the teams formation by 18th Aug (tentative deadline). Same teams will be working towards project and other 2 Assignments as well. 
 


# Q1: Transformations and Projections on Autonomous Driving Dataset (20 Points)

In this question, you will work with real world autonomous driving dataset (sequence in Waymo dataset). The dataset has LiDAR point clouds, images. You are required to demonstrate: 

**I. Various transformations of rotation matrices as described in below tasks.**

**II. Visualization as a result of above transformations in Open3D**

## Given data:

1.) `LiDAR Point Clouds` : Stored at each timestep in the folder `lidar`. The point clouds are provided in the ego frame attached to lidar sensor (vehicle's reference frame).

2.) `Images` : Stored at each timestep in the folder `images`. 

**Naming Convention** : {timestep}_{cam_no}.jpg where timestep is specified in 3 digits and cam_no : [0, 1, 2] indicates centre, left and right camera respectively.

3.) `Camera-to-Ego Transformations`: Stored in the folder `cam2ego`, which converts points from each camera's reference frame to the vehicle's (or ego) reference frame.

4.) `Ego-to-World Transformations`: Stored in the folder `ego2world`, which converts points from the vehicle's reference frame to the world frame W.

5.) `Camera Intrinsics`: Stored in the folder `intrinsics` provided for 3 cameras.




### Helper functions to read lidar data and camera instrinsics are provided below

In [1]:
# Helper function to read instrinsic matrix

import numpy as np
import os 

def read_intrinsic(timestep):
    intrinsic = np.loadtxt(f"sample_intrinsic_{timestep}.txt")
    fx, fy, cx, cy = intrinsic[0], intrinsic[1], intrinsic[2], intrinsic[3]
    intrinsic_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

read_intrinsic(1)

In [2]:
# Helper function to read lidar data at timestep 0 (same logic to read lidars at all remaining timesteps)

lidar_data = np.memmap('sample_lidar_data_000.bin',
                dtype=np.float32,
                mode="r",
            ).reshape(-1, 14)   # (165454, 14)

lidar_origins = lidar_data[:, :3]
lidar_points = lidar_data[:, 3:6]   # (165454, 3)
lidar_ids = lidar_data[:, -1]   # (165454,)

# Lidar points to be used 
print(lidar_points.shape)

(165454, 3)


In [1]:
import os
import zipfile
import gdown
import shutil
import numpy as np
import open3d as o3d
import math
import cv2

def get_ego2world(timestep):
    filename = f'ego2world/{timestep:03d}.txt'
    transform = np.loadtxt(filename)
    transform = transform.reshape(4, 4)
    return transform

def get_cam2ego(camera_id):
    filename = f'cam2ego/{camera_id}.txt'
    transform = np.loadtxt(filename)
    transform = transform.reshape(4, 4)
    return transform

def get_intrinsics(camera_id):
    filename = f'intrinsics/{camera_id}.txt'
    intrinsics = np.loadtxt(filename)
    intrinsic_matrix = np.array([[intrinsics[0], 0, intrinsics[2]], [0, intrinsics[1], intrinsics[3]], [0, 0, 1]])
    return intrinsic_matrix

def get_image_path(camera_id,timestep):
    filename = f'images/{timestep:03d}_{camera_id}.jpg'
    return filename

def get_lidar_points(timestep):
    filename = f'lidar/{timestep:03d}.bin'
    lidar_data = np.memmap(filename,
                dtype=np.float32,
                mode="r",
            ).reshape(-1, 14)   # (165454, 14)
    lidar_points = lidar_data[:, 3:6]   # (165454, 3)
    return lidar_points

def transform_vector(vector, transformation_matrix):
    homogeneous_vector = np.append(vector, 1)
    
    transformed_vector = np.matmul(transformation_matrix, homogeneous_vector)

    return transformed_vector[:3]/transformed_vector[3]

def transform_point_cloud(point_cloud,transformation_matrix,downsample = 1.0):
    transformed_point_cloud = np.zeros((int(point_cloud.shape[0] * downsample), 3))
    for i in range(int(point_cloud.shape[0] * downsample)):
        transformed_point_cloud[i] = transform_vector(point_cloud[int(i/downsample)],transformation_matrix)
    return transformed_point_cloud

def invert_transform(matrix):
    rotation_matrix = matrix[:3, :3]
    translation_vector = matrix[:3, 3]

    inv_rotation_matrix = np.transpose(rotation_matrix)
    inv_translation_vector = -np.matmul(inv_rotation_matrix, translation_vector)

    inv_matrix = np.eye(4)
    inv_matrix[:3, :3] = inv_rotation_matrix
    inv_matrix[:3, 3] = inv_translation_vector
    
    return inv_matrix

def concatenate_point_clouds(func_get_single_pcl,func_get_associated_transform,start_timestep,stop_timestep,downsample = 1.0):
    concatenated_point_cloud = np.empty((0, 3))
    for i in range(start_timestep, stop_timestep + 1):
        concatenated_point_cloud = np.vstack([concatenated_point_cloud,
            transform_point_cloud(func_get_single_pcl(i),func_get_associated_transform(i),downsample)]
        )
    return concatenated_point_cloud

def visualize_points(lidar_points):
    # Create Open3D point cloud object from numpy array
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(lidar_points)
    o3d.visualization.draw_geometries([point_cloud])

def alpha_rot(alpha):
    return np.array([
        [   1.0,    0.0,                0.0             ],
        [   0.0,    math.cos(alpha),   -math.sin(alpha) ],
        [   0.0,    math.sin(alpha),    math.cos(alpha) ]
        ],dtype=np.float32)

def beta_rot(beta):
    return np.array([
        [   math.cos(beta),     0.0,    math.sin(beta)  ],
        [   0.0,                1.0,    0.0             ],
        [  -math.sin(beta),     0.0,    math.cos(beta)  ]
        ],dtype=np.float32)

def gamma_rot(gamma):
    return np.array([
        [   math.cos(gamma),   -math.sin(gamma),    0.0 ],
        [   math.sin(gamma),    math.cos(gamma),    0.0 ],
        [   0.0,                0.0,                1.0 ]
        ],dtype=np.float32)
    
def map_value_to_color(value, min_val, max_val, color1, color2):
    normalized_value = (value - min_val) / (max_val - min_val)
    color = (1 - normalized_value) * color1 + normalized_value * color2
    return color.astype(np.uint8)

def skew(vector):
    return np.array([
        [ 0,         -vector[2],  vector[1]  ],
        [ vector[2],  0,         -vector[0]  ],
        [-vector[1],  vector[0],  0          ]])

def prepare_data():
    # List of expected extracted folders
    expected_folders = ["cam2ego", "ego2world", "images", "intrinsics", "lidar"]

    # Flag to check if any folder is missing or empty
    should_download = False

    # Check if all folders exist and are not empty
    for folder in expected_folders:
        if not os.path.exists(folder) or not os.listdir(folder):
            should_download = True
            break

    if should_download:
        # Download the zip file using gdown
        gdown.download(id="1HKQ6UGFlgkRpIZzZWs6GkVtXdCGKzT3Z", output="data.zip", quiet=False)

        # Extract the contents of the zip file into a specific directory
        with zipfile.ZipFile("data.zip", 'r') as zip_ref:
            zip_ref.extractall(".")

        # Define the extracted folder
        extracted_folder = "080_MR"

        # Move the contents of the extracted folder to the current directory
        for item in os.listdir(extracted_folder):
            s = os.path.join(extracted_folder, item)
            d = os.path.join(".", item)
            if os.path.isdir(s):
                shutil.move(s, d)
            else:
                shutil.move(s, d)

        # Delete the now empty extracted folder
        os.rmdir(extracted_folder)

        # Delete the downloaded zip file if no longer needed
        os.remove("data.zip")
        # Get Concatened PCl ready for tasks 
    else:
        print("All folders already exist and are not empty. Skipping download and extraction.")
    global conc_pcl
    if not os.path.exists("conc_pcl.npy"):
        conc_pcl = concatenate_point_clouds(get_lidar_points, get_ego2world, 0, 197)
        np.save('conc_pcl',conc_pcl)
    else:
        print("conc_pcl is already populated. Skipping concatenation.")
    conc_pcl = np.load('conc_pcl.npy')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
prepare_data()

All folders already exist and are not empty. Skipping download and extraction.
conc_pcl is already populated. Skipping concatenation.



**Note:** Even though Waymo dataset has 5 cameras, you are given the dataset corresponding to middle 3 cameras only. Please ignore other 2 cameras.

## Notation for tasks:

a.) `Global Reference Frame G`: Defined as the first ego frame (i.e., the translation vector of ego2world[0] is the origin of frame G in world frame W).

World Frame W: A fixed world reference frame.

b.) `Ego Frame`: Attached to the LiDAR and changes as the vehicle moves.

c.) `Camera Frames`: Each of the 5 cameras has its own frame, which changes as the vehicle moves.

Note: Axis directions of `Ego Frame` and `Camera Frames` are aligned with the Waymo Coordinate System (LiDAR) described below

## Coordinate Systems:

**OpenCV Coordinate System:** x right, y down, z front.

**Waymo Coordinate System (LiDAR):** x front, y left, z up.


![Waymo Setup](./waymo_setup.jpg "Waymo Setup")

Link to dataset (one sequence) : https://drive.google.com/drive/folders/17YDx2Yn1KmPjmlaHsoFz4Jpa8zpgovO2?usp=drive_link

If you want to try on other sequences as well, please refer to : https://waymo.com/open/

### `Task 1`. Transformations of LiDAR Point Clouds (10 points)

**Instructions:** 

Transform the LiDAR point clouds at all timesteps to the global reference frame G. Concatenate these transformed point clouds.
    
Visualization: Use Open3D to visualize the concatenated point cloud in the global reference frame G. Also, display the concatenation process at every timestep starting from first point cloud





In [1]:
##############################################################################
# TODO: TASK 1
##############################################################################
vis = o3d.visualization.Visualizer()
vis.create_window()
# Create Open3D point cloud object from numpy array
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(transform_point_cloud(get_lidar_points(0),get_ego2world(0),0.1))
vis.add_geometry(point_cloud)

try:
    for i in range(197):
        point_cloud.points.extend(o3d.utility.Vector3dVector(transform_point_cloud(get_lidar_points(i+1), get_ego2world(i+1), 0.1)))
        vis.update_geometry(point_cloud)
        vis.poll_events()
        vis.update_renderer()
except KeyboardInterrupt:
    print("Visualization interrupted. Closing the window...")

# Safe exiting on button press or keyboard interrupt
while True:
    try:
        vis.poll_events()
        vis.update_renderer()
    except KeyboardInterrupt:
        print("Visualization interrupted. Closing the window...")
        break

# Destroy the window properly
vis.destroy_window()


NameError: name 'o3d' is not defined

### `Task 2`. Projecting LiDAR Point Clouds onto images (10 points)

**Instructions:**

Transform the concatenated point cloud from task 1 to the frame of each of the 3 cameras at timesteps `0, 20, and 55`. 
    
Project these transformed point clouds onto the respective camera frames using the provided camera intrinsics. Concatenated point cloud would be very dense, so randomly select arbitrary number of points for better visualization. 

**Projected image pixel x : K * X_3d where X_3d is the 3d point in camera frame.**
    
Visualization: Overlay the projected points onto the camera images and visualize them.

**For example:** Overlayed concatenated point cloud on camera `000_0.png` and `030_2.png` are shown below

<table><tr>
<td> <img src="./projected_000_0.png" alt="Drawing" style="width: 750px;"/> </td>
<td> <img src="./projected_030_2.png" alt="Drawing" style="width: 750px;"/> </td>
</tr></table>



In [12]:
##############################################################################
# TODO: TASK 2
##############################################################################
def task2(cam,timesteps,resolution = 1.0):    
    cam2cv = np.array([
        [0.0,-1.0, 0.0, 0.0],
        [0.0, 0.0, -1.0, 0.0],
        [1.0, 0.0,0.0, 0.0],
        [0.0, 0.0, 0.0, 1.0]
    ], dtype=np.float32)
    intrinsics = get_intrinsics(cam)
    ego2cam = invert_transform(get_cam2ego(cam))
    world2ego = invert_transform(get_ego2world(timesteps))

    camera_frame_point_cloud = transform_point_cloud(conc_pcl,np.matmul(cam2cv,np.matmul(ego2cam,world2ego)),resolution)

    pixels = np.empty((camera_frame_point_cloud.shape[0], 3))
    for i in range(camera_frame_point_cloud.shape[0]):
        pixels[i] = np.matmul(intrinsics,camera_frame_point_cloud[i])
        pixels[i] /= pixels[i][2]
    image = cv2.imread(get_image_path(cam,timesteps))
    for i in range(pixels.shape[0]):
        x, y = np.round(pixels[i][:2]).astype(int)
        if 0 <= x < image.shape[1] and 0 <= y < image.shape[0]:
            cv2.circle(image, (x, y), radius=2, color=(0, 255, 0), thickness=-1)

    cv2.namedWindow('Resizable Window', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Resizable Window', 400, 300)
    cv2.imshow('Resizable Window', image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:

task2(0,0,0.01)
task2(2,30,0.01)


## Bonus

### `Task 3`. Compute Depth Image from Projected Point Cloud in camera frame (5 points)

**Instructions:**

Using the projected point clouds to camera frame from task 2, visualize the depth image by considering only the z-coordinate of the projected points in the camera frame.

Visualization: Display the depth image for each of the 3 cameras at timesteps `0, 20, and 55` alongside the corresponding RGB image.

In [18]:
##############################################################################
# TODO: TASK 3
##############################################################################
def task3(cam,timesteps,resolution = 1.0):
    cam2cv = np.array([
        [0.0,-1.0, 0.0, 0.0],
        [0.0, 0.0, -1.0, 0.0],
        [1.0, 0.0,0.0, 0.0],
        [0.0, 0.0, 0.0, 1.0]
    ], dtype=np.float32)
    intrinsics = get_intrinsics(cam)
    ego2cam = invert_transform(get_cam2ego(cam))
    world2ego = invert_transform(get_ego2world(timesteps))

    camera_frame_point_cloud = transform_point_cloud(conc_pcl,np.matmul(cam2cv,np.matmul(ego2cam,world2ego)),resolution)

    pixels = np.empty((camera_frame_point_cloud.shape[0], 3))
    for i in range(camera_frame_point_cloud.shape[0]):
        pixels[i] = np.matmul(intrinsics,camera_frame_point_cloud[i])
        pixels[i] /= pixels[i][2]
    image = np.ones((1280, 1920), dtype=np.uint8) * 255
    for i in range(camera_frame_point_cloud.shape[0]):
        x, y = np.round(pixels[i][:2]).astype(int)
        if 0 <= y < image.shape[0] and 0 <= x < image.shape[1]:
            if((image[y,x])>(camera_frame_point_cloud[i][2]*10.0)):
                image[y,x] = (camera_frame_point_cloud[i][2]*10)

    cv2.namedWindow('Resizable Window', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Resizable Window', 400, 300)
    cv2.imshow('Resizable Window', image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [20]:
task3(0,0,0.1)
task3(1,0)
task3(2,0)
task3(0,20)
task3(1,20)
task3(2,20)
task3(0,55)
task3(1,55)
task3(2,55)

#### Note: You might be asked to show the above results for different timesteps and from one of the 3 cameras during evaluation/viva.

# Q2: Various Representations for Rotations and Gimbal lock (15 points)


#### 2.1 Euler angles (2.5 points)

a. Write a function that returns a rotation matrix given the angles (𝛼, 𝛽, 𝛾) = (2π/5, π/18, π/6) in radians (X-Y-Z). Do not use inbuilt functions.

b. Solve for angles using fsolve from scipy for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 &  0.944799  \\0.21984631 &  0.96542533  & 0.14007684 \\
   -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

In [11]:
##############################################################################
# TODO: Do tasks described in 2.1 (a)
##############################################################################
def euler2rm(alpha, beta, gamma):
    return np.matmul(alpha_rot(alpha),np.matmul(beta_rot(beta),gamma_rot(gamma)))
    

In [18]:
alpha = (2*math.pi)/5.0
beta = (math.pi)/18.0
gamma = (math.pi)/6.0
print(euler2rm(alpha,beta,gamma))

[[ 0.8528685  -0.49240386  0.17364818]
 [ 0.29753193  0.18504195 -0.93660784]
 [ 0.42905715  0.85046923  0.30432233]]


In [16]:
##############################################################################
# TODO: Do tasks described in 2.1 (b)
##############################################################################
target_matrix = np.array([
    [0.26200263, -0.19674724, 0.944799  ],
    [0.21984631,  0.96542533, 0.14007684],
    [-0.93969262, 0.17101007, 0.29619813],
    ], dtype=np.float32)
def f(euler,rm):
    alpha,beta,gamma = euler
    residual_matrix  = euler2rm(alpha,beta,gamma) - rm
    return np.sum(np.abs(residual_matrix), axis=1)

from scipy.optimize import fsolve
x = fsolve(f,[0,2,0],args = target_matrix)
print(x)

[ 0.55414196  1.31585304 -0.31566263]


/tmp/ipykernel_225429/3350393974.py:15: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  x = fsolve(f,[0,2,0],args = target_matrix)


#### 2.2 Equivalent angle–axis representation (2.5 points) 

 Write a function to convert equivalent angle–axis representation (with a general axis and angle) to matrix form and vice versa. \
Try it for $\theta = \pi/6$ and axis $K= [1, 2, 3]^T $

In [19]:
##############################################################################
# TODO: Do tasks described in 2.2 
##############################################################################

def axisangle2rm(theta, axis):
    return ((1 - np.cos(theta))*(np.matmul(np.atleast_2d(axis).transpose(),np.atleast_2d(axis)))) + (np.cos(theta)*np.eye(3)) + np.sin(theta)*skew(axis)

print(axisangle2rm(np.pi/6,np.array([1,2,3])))

[[ 1.         -1.23205081  1.40192379]
 [ 1.76794919  1.40192379  0.30384758]
 [-0.59807621  1.30384758  2.07179677]]


#### 2.3 Gimbal lock (5 points)

Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given point cloud, data/toothless.ply. You have to show the above by animation (rotation along each axis one by one).

**Hint:** 
Create 3 disks perpendicular to each other representing axes for local frame of object. Show that in certain configuration, due to use of Euler angles we can lose a degree of freedom. 

Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by 20° around a particular axis, do so in increments of 5° 4 times to make it look like an animation.

In [8]:
import open3d as o3d
import numpy as np
from datetime import datetime, timedelta

# Load the point cloud
pcd = o3d.io.read_point_cloud("toothless.ply")

# Create the axes disks
def create_disk(radius=0.1, axis=[1, 0, 0], resolution=30):
    return o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=0.01, resolution=resolution).rotate(
        o3d.geometry.get_rotation_matrix_from_xyz(np.pi/2 * np.array(axis))
    ).translate(np.array(axis) * radius)

# Create the three disks representing the axes
disk_x = create_disk(axis=[1, 0, 0])
disk_y = create_disk(axis=[0, 1, 0])
disk_z = create_disk(axis=[0, 0, 1])

# Combine the disks with the point cloud
geometry = [pcd, disk_x, disk_y, disk_z]

# Create the visualizer
vis = o3d.visualization.Visualizer()
vis.create_window()

for geom in geometry:
    vis.add_geometry(geom)

# Function to apply rotation and simulate animation
def rotate_and_visualize(vis, geometry, axis, angle_deg):
    axis = np.array(axis)  # Convert the axis to a NumPy array
    angle_rad = np.deg2rad(angle_deg)
    R = o3d.geometry.get_rotation_matrix_from_xyz(axis * angle_rad)
    for geom in geometry:
        geom.rotate(R, center=(0, 0, 0))
        vis.update_geometry(geom)  # Update each geometry object
    vis.poll_events()
    vis.update_renderer()

# Simulate rotation animation
angles = np.arange(0, 90 + 1, 1)  # Rotate 90 degrees in 5 degree steps

for angle in angles:
    rotate_and_visualize(vis, geometry, axis=[1, 0, 0], angle_deg=5)

for angle in angles:
    rotate_and_visualize(vis, geometry, axis=[0, 1, 0], angle_deg=5)

for angle in angles:
    rotate_and_visualize(vis, geometry, axis=[0, 0, 1], angle_deg=5)

# Simulate Gimbal lock by rotating 90 degrees on Y and then 90 degrees on X
rotate_and_visualize(vis, geometry, axis=[0, 1, 0], angle_deg=90)
rotate_and_visualize(vis, geometry, axis=[1, 0, 0], angle_deg=90)

# Here, rotating around the Z-axis won't change the orientation due to Gimbal lock
    rotate_and_visualize(vis, geometry, axis=[0, 0, 1], angle_deg=5)

# Get the current timestamp
timestamp = datetime.now()

# Run the loop for 10 seconds
while (datetime.now() - timestamp) < timedelta(seconds=10):
    vis.poll_events()
    vis.update_renderer()

vis.destroy_window()


#### 2.4: Quaternions (5 points)

a. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.

b. Perform matrix multiplication of two 3×3 rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both cases is the same.

c. Try to interpolate any given model between two rotation matrices and visualize!

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.4 (a)
##############################################################################

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.4 (b)
##############################################################################

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.4 (c)
##############################################################################

# Q3: Interpolation between transformations (15 points)

Given 2 random transformation matrices, interpolate the given point cloud **toothless.ply** from `T2` to `T1` and visualize it.

We will use the `generateTransformation()` function to generate a random Transformation matrix. You can write your own `generateTransformation()` function for testing, but we will replace it with our own so make sure that your code works for general cases.

Ensure that your visualization shows the starting and ending configurations during interpolation.

Your final output should look something like this:
![Visualization](./out.gif)

In [ ]:
def generateTransformation():
    pass

In [ ]:
T1 = generateTransformation()
T2 = generateTransformation()

## Bonus (5 points)

In [ ]:
##############################################################################
# TODO: Implement the above question using spherical linear interpolation (slerp)
##############################################################################

#### References: https://en.wikipedia.org/wiki/Slerp